# **Importing the required libraries**

In [1]:
import numpy as np 
import pandas as pd
import os
import torch
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
# import torchvision.transforms.functional as TF

import random
import os, shutil
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from os.path import join
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
import cv2

import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, sampler
from albumentations import (HorizontalFlip, VerticalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# **Data Loading and Decoding**

In [2]:
data_train = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')

In [3]:
data_train.head()

In [4]:
DATA_PATH = '../input/sartorius-cell-instance-segmentation'
SAMPLE_SUBMISSION = join(DATA_PATH,'train')
TRAIN_CSV = join(DATA_PATH,'train.csv')
TRAIN_PATH = join(DATA_PATH,'train')
TEST_PATH = join(DATA_PATH,'test')

df_train = pd.read_csv(TRAIN_CSV)
print(f'Training Set Shape: {df_train.shape} - {df_train["id"].nunique()} \
Images - Memory Usage: {df_train.memory_usage().sum() / 1024 ** 2:.2f} MB')

In [5]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)
def build_masks(df_train, image_id, input_shape):
    height, width = input_shape
    labels = df_train[df_train["id"] == image_id]["annotation"].tolist()
    mask = np.zeros((height, width))
    for label in labels:
        mask += rle_decode(label, shape=(height, width))
    mask = mask.clip(0, 1)
    return np.array(mask)


In [6]:
class CellDataset(Dataset):
    def __init__(self, df: pd.core.frame.DataFrame, train:bool):
        self.IMAGE_RESIZE = (224, 224)
        self.RESNET_MEAN = (0.485, 0.456, 0.406)
        self.RESNET_STD = (0.229, 0.224, 0.225)
        self.df = df
        self.base_path = TRAIN_PATH
        self.gb = self.df.groupby('id')
        self.transforms = Compose([Resize(self.IMAGE_RESIZE[0],  self.IMAGE_RESIZE[1]),
                                   Normalize(mean=self.RESNET_MEAN, std= self.RESNET_STD, p=1),
                                   HorizontalFlip(p=0.5),
                                   VerticalFlip(p=0.5)])
        
        # Split train and val set
        all_image_ids = np.array(df_train.id.unique())
        np.random.seed(42)
#         iperm = np.random.permutation(len(all_image_ids))
        num_train_samples = int(len(all_image_ids) * 0.9)

        if train:
            self.image_ids = all_image_ids[:num_train_samples]
        else:
             self.image_ids = all_image_ids[num_train_samples:]

    def __getitem__(self, idx: int) -> dict:

        image_id = self.image_ids[idx]
        df = self.gb.get_group(image_id)

        # Read image
        image_path = os.path.join(self.base_path, image_id + ".png")
        image = cv2.imread(image_path)

        # Create the mask
        mask = build_masks(df_train, image_id, input_shape=(520, 704))
        mask = (mask >= 1).astype('float32')
        augmented = self.transforms(image=image, mask=mask)
        image = augmented['image']
        mask = augmented['mask']
        # print(np.moveaxis(image,0,2).shape)
        return np.moveaxis(np.array(image),2,0), mask.reshape((1, self.IMAGE_RESIZE[0], self.IMAGE_RESIZE[1]))


    def __len__(self):
        return len(self.image_ids)

In [7]:
ds_train = CellDataset(df_train, train=True)
dl_train = DataLoader(ds_train, batch_size=16, num_workers=2, pin_memory=True, shuffle=False)

In [8]:
ds_test = CellDataset(df_train, train=False)
dl_test = DataLoader(ds_test, batch_size=4, num_workers=2, pin_memory=True, shuffle=False)


# **Visualization of the images and masks**

In [9]:
def imshow(num_to_show=9):
    
    plt.figure(figsize=(20,20))
    
    for i in range(num_to_show):
        plt.subplot(3, 3, i+1)
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
        
        img = mpimg.imread(f'../input/sartorius-cell-instance-segmentation/train/{data_train.iloc[i*500,0]}.png')
        plt.imshow(img, cmap='plasma')

imshow()

In [10]:
# plot simages and mask from dataloader
batch = next(iter(dl_train))
images, masks = batch
print(f"image shape: {images.shape},\nmask shape:{masks.shape},\nbatch len: {len(batch)}")
k=11
plt.figure(figsize=(20, 20))
        
plt.subplot(1, 3, 1)
plt.xticks([])
plt.yticks([])
plt.imshow(images[k][0])
plt.title('Original image')

plt.subplot( 1, 3, 2)
plt.xticks([])
plt.yticks([])
plt.imshow(masks[k][0])
plt.title('Mask')

plt.subplot( 1, 3, 3)
plt.xticks([])
plt.yticks([])
plt.imshow(images[k][0])
plt.imshow(masks[k][0],alpha=0.2)
plt.title('Both')
plt.tight_layout()
plt.show()

# K-means

In [ ]:
def testKmeans(imageIndex):

    shape = (520, 704, 3)
    data = rle.loadData()
    lens = 0
    for i in range(len(data['annotation'][imageIndex])):
        if i == 0:
            mask = rle.rle_decode(data['annotation'][imageIndex][i], shape)
            lens = len(data['annotation'][imageIndex][i])
        else:
            mask = cv2.add(mask, rle.rle_decode(data['annotation'][imageIndex][i], shape))
            lens += len(data['annotation'][imageIndex][i])
        fileName = data['image_path'][imageIndex]
    print('annnotation len', lens)
    img = cv2.imread(fileName)
    mask *= 255.0
    print('mask_shape', mask.shape)
    mask = mask.astype(np.uint8)

    mask_img = cv2.addWeighted(img, 1, mask, 0.2, 0)

    cv2.imshow('origin', img)
    cv2.imshow('mask', mask)
    cv2.imshow('mask img', mask_img)
    cv2.waitKey(0)

    mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
    flat_mask = mask.flatten()

    return mask, flat_mask
class RLE:
    def __init__(self):
        return
    def mask2rle(self, img):
        '''
        Efficient implementation of mask2rle, from @paulorzp
        --
        img: numpy array, 1 - mask, 0 - background
        Returns run length as string formated
        Source: https://www.kaggle.com/xhlulu/efficient-mask2rle
        '''
        pixels = img.T.flatten()
        pixels = np.pad(pixels, ((1, 1),))
        runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
        runs[1::2] -= runs[::2]
        return ' '.join(str(x) for x in runs)


    def rle_decode(self, mask_rle, shape, color=1):
        '''
        mask_rle: run-length as string formated (start length)
        shape: (height, width, channels) of array to return
        color: color for the mask
        Returns numpy array (mask)

        '''
        s = mask_rle.split()
        starts = list(map(lambda x: int(x) - 1, s[0::2]))
        lengths = list(map(int, s[1::2]))
        ends = [x + y for x, y in zip(starts, lengths)]

        img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)

        for start, end in zip(starts, ends):
            img[start: end] = color

        return img.reshape(shape)
rle = RLE()

# **CNN**

In [11]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.cov1=nn.Conv2d(in_channels, 20, kernel_size=5, padding="same")
        self.btn=nn.BatchNorm2d(20)
        self.relu=nn.ReLU()
        self.cov2=nn.Conv2d(20, 10, kernel_size=1)
        self.cov3=nn.Conv2d(10, 10, kernel_size=5, padding="same")
        self.btn2=nn.BatchNorm2d(10)
        self.cov4=nn.Conv2d(10, num_classes, kernel_size=1)
        self.sigmod=nn.Sigmoid()
        
    def forward(self, x):
        # print(x.shape)
        x1 = self.cov1(x)
        # print(x1.shape)
        x2 = self.btn(x1)
        # print(x2.shape)
        x3 = self.relu(x2)
        # print(x3.shape)
        x4 = self.cov2(x3)
        # print(x4.shape)
        x5 = self.cov3(x4)
        # print(x5.shape)
        # print('up')
        x = self.btn2(x4)
        # print(x.shape)
        x = self.relu(x)
        # print(x.shape)
        x = self.cov4(x)
        # print(x.shape)
        x = self.sigmod(x)
        # print(x.shape)
        return x

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device=torch.device('cpu')

In [13]:
def train_loop(model, optimizer, criterion, train_loader, device=device):
    running_loss = 0
    model.train()
    pbar = tqdm(train_loader, desc='Iterating over train data')
    for imgs, masks in pbar:
        # pass to device
        imgs = imgs.to(device)
        masks = masks.to(device)
        # forward
        out = model(imgs)
        loss = criterion(out, masks)
        running_loss += loss.item()*imgs.shape[0]  # += loss * current batch size
        # optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    running_loss /= len(train_loader.sampler)
    return running_loss

In [14]:
def eval_loop(model, criterion, eval_loader, device=device):
    running_loss = 0
    model.eval()
    with torch.no_grad():
        accuracy, f1_scores = [], []
        pbar = tqdm(eval_loader, desc='Iterating over evaluation data')
        
        for imgs, masks in pbar:
#             print(imgs.shape)
#             print(masks.shape)
            # pass to device
            li=imgs
            lm=masks
            imgs = imgs.to(device)
            masks = masks.to(device)
            # forward
            out = model(imgs)
#             print(out.shape)
            loss = criterion(out, masks)
            running_loss += loss.item()*imgs.shape[0]
            # calculate predictions using output
            predicted = (out > 0.5).float()
#             print(predicted.shape)
            predicted = predicted.view(-1).cpu().numpy()
            labels = masks.view(-1).cpu().numpy()
#             print(predicted.shape)
#             print(labels.shape)
            accuracy.append(accuracy_score(labels, predicted))
            f1_scores.append(f1_score(labels, predicted))
    acc = sum(accuracy)/len(accuracy)
    f1 = sum(f1_scores)/len(f1_scores)
    running_loss /= len(eval_loader.sampler)
    return {
        'accuracy':acc,
        'f1_macro':f1, 
        'loss':running_loss,
        'img': li,
        'masks': lm,
        'out':out
        
    }

In [15]:
def train(model, optimizer, criterion, train_loader, valid_loader,
          device=device, 
          num_epochs=25, 
          valid_loss_min=np.inf,
          logdir='logdir'):
    
    tb_writer = SummaryWriter(log_dir=logdir)
    for e in range(num_epochs):
        # train for epoch
        train_loss = train_loop(
            model, optimizer, criterion, train_loader, device=device)
        # evaluate on validation set
        metrics = eval_loop(
            model, criterion, valid_loader, device=device
        )
        # show progress
        print_string = f'Epoch: {e+1} '
        print_string+= f'TrainLoss: {train_loss:.5f} '
        print_string+= f'ValidLoss: {metrics["loss"]:.5f} '
        print_string+= f'ACC: {metrics["accuracy"]:.5f} '
        print_string+= f'F1: {metrics["f1_macro"]:.3f}'
        print(print_string)

        # Tensorboards Logging
        tb_writer.add_scalar('UNet/Train Loss', train_loss, e)
        tb_writer.add_scalar('UNet/Valid Loss', metrics["loss"], e)
        tb_writer.add_scalar('UNet/Accuracy', metrics["accuracy"], e)
        tb_writer.add_scalar('UNet/F1 Macro', metrics["f1_macro"], e)

        # save the model 
        if metrics["loss"] <= valid_loss_min:
            torch.save(model.state_dict(), 'model.pt')
            valid_loss_min = metrics["loss"]

In [16]:
# set_seed(21)
model1 = CNN(3, 1).to(device)
optimizer = optim.Adam(model1.parameters(), lr=0.01)
criterion = nn.BCELoss()
train(model1, optimizer, criterion, dl_train, dl_test)

In [17]:
# Load the latest model
model1.load_state_dict(torch.load('model.pt'))
metrics = eval_loop(model1, criterion, dl_test)

print('accuracy:', metrics['accuracy'])
print('f1 macro:', metrics['f1_macro'])
print('test loss:', metrics['loss'])

# Unet

In [18]:
class conv_block(nn.Module):
    """
    Convolution Block 
    """
    def __init__(self, in_ch, out_ch):
        super(conv_block, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True))

    def forward(self, x):

        x = self.conv(x)
        return x


class up_conv(nn.Module):
    """
    Up Convolution Block
    """
    def __init__(self, in_ch, out_ch):
        super(up_conv, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.up(x)
        return x

In [19]:
class U_Net(nn.Module):
    """
    UNet - Basic Implementation
    Paper : https://arxiv.org/abs/1505.04597
    """
    def __init__(self, in_ch=3, out_ch=1):
        super(U_Net, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]
        
        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(in_ch, filters[0])
        self.Conv2 = conv_block(filters[0], filters[1])
        self.Conv3 = conv_block(filters[1], filters[2])
        self.Conv4 = conv_block(filters[2], filters[3])
        self.Conv5 = conv_block(filters[3], filters[4])

        self.Up5 = up_conv(filters[4], filters[3])
        self.Up_conv5 = conv_block(filters[4], filters[3])

        self.Up4 = up_conv(filters[3], filters[2])
        self.Up_conv4 = conv_block(filters[3], filters[2])

        self.Up3 = up_conv(filters[2], filters[1])
        self.Up_conv3 = conv_block(filters[2], filters[1])

        self.Up2 = up_conv(filters[1], filters[0])
        self.Up_conv2 = conv_block(filters[1], filters[0])

        self.Conv = nn.Conv2d(filters[0], out_ch, kernel_size=1, stride=1, padding=0)

        self.active = torch.nn.Sigmoid()

    def forward(self, x):

        e1 = self.Conv1(x)

        e2 = self.Maxpool1(e1)
        e2 = self.Conv2(e2)

        e3 = self.Maxpool2(e2)
        e3 = self.Conv3(e3)

        e4 = self.Maxpool3(e3)
        e4 = self.Conv4(e4)

        e5 = self.Maxpool4(e4)
        e5 = self.Conv5(e5)

        d5 = self.Up5(e5)
        d5 = torch.cat((e4, d5), dim=1)

        d5 = self.Up_conv5(d5)

        d4 = self.Up4(d5)
        d4 = torch.cat((e3, d4), dim=1)
        d4 = self.Up_conv4(d4)

        d3 = self.Up3(d4)
        d3 = torch.cat((e2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((e1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        out = self.Conv(d2)

        out = self.active(out)

        return out

In [20]:
# set_seed(21)
model2 = U_Net(3, 1).to(device)
optimizer = optim.Adam(model2.parameters(), lr=0.01)
criterion = nn.BCELoss()
train(model2, optimizer, criterion, dl_train, dl_test)

In [21]:
# Load the latest model
model2.load_state_dict(torch.load('model.pt'))
metrics = eval_loop(model2, criterion, dl_test)

print('accuracy:', metrics['accuracy'])
print('f1 macro:', metrics['f1_macro'])
print('test loss:', metrics['loss'])

# Attention Unet

In [22]:
class Attention_block(nn.Module):
    """
    Attention Block
    """

    def __init__(self, F_g, F_l, F_int):
        super(Attention_block, self).__init__()

        self.W_g = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        out = x * psi
        return out


class AttU_Net(nn.Module):
    """
    Attention Unet implementation
    Paper: https://arxiv.org/abs/1804.03999
    """
    def __init__(self, img_ch=3, output_ch=1):
        super(AttU_Net, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(img_ch, filters[0])
        self.Conv2 = conv_block(filters[0], filters[1])
        self.Conv3 = conv_block(filters[1], filters[2])
        self.Conv4 = conv_block(filters[2], filters[3])
        self.Conv5 = conv_block(filters[3], filters[4])

        self.Up5 = up_conv(filters[4], filters[3])
        self.Att5 = Attention_block(F_g=filters[3], F_l=filters[3], F_int=filters[2])
        self.Up_conv5 = conv_block(filters[4], filters[3])

        self.Up4 = up_conv(filters[3], filters[2])
        self.Att4 = Attention_block(F_g=filters[2], F_l=filters[2], F_int=filters[1])
        self.Up_conv4 = conv_block(filters[3], filters[2])

        self.Up3 = up_conv(filters[2], filters[1])
        self.Att3 = Attention_block(F_g=filters[1], F_l=filters[1], F_int=filters[0])
        self.Up_conv3 = conv_block(filters[2], filters[1])

        self.Up2 = up_conv(filters[1], filters[0])
        self.Att2 = Attention_block(F_g=filters[0], F_l=filters[0], F_int=32)
        self.Up_conv2 = conv_block(filters[1], filters[0])

        self.Conv = nn.Conv2d(filters[0], output_ch, kernel_size=1, stride=1, padding=0)

        self.active = torch.nn.Sigmoid()


    def forward(self, x):

        e1 = self.Conv1(x)

        e2 = self.Maxpool1(e1)
        e2 = self.Conv2(e2)

        e3 = self.Maxpool2(e2)
        e3 = self.Conv3(e3)

        e4 = self.Maxpool3(e3)
        e4 = self.Conv4(e4)

        e5 = self.Maxpool4(e4)
        e5 = self.Conv5(e5)

        #print(x5.shape)
        d5 = self.Up5(e5)
        #print(d5.shape)
        x4 = self.Att5(g=d5, x=e4)
        d5 = torch.cat((x4, d5), dim=1)
        d5 = self.Up_conv5(d5)

        d4 = self.Up4(d5)
        x3 = self.Att4(g=d4, x=e3)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.Up_conv4(d4)

        d3 = self.Up3(d4)
        x2 = self.Att3(g=d3, x=e2)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        x1 = self.Att2(g=d2, x=e1)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        out = self.Conv(d2)
#         print(out.shape)

        out = self.active(out)

        return out

In [23]:
# set_seed(21)
model3 = AttU_Net(3, 1).to(device)
optimizer = optim.Adam(model3.parameters(), lr=0.01)
criterion = nn.BCELoss()
train(model3, optimizer, criterion, dl_train, dl_test)

In [24]:
# Load the latest model
model3.load_state_dict(torch.load('model.pt'))
metrics = eval_loop(model3, criterion, dl_test)

print('accuracy:', metrics['accuracy'])
print('f1 macro:', metrics['f1_macro'])
print('test loss:', metrics['loss'])

# **comparison of 3 supervised learning methods**

In [25]:
batchs = next(iter(dl_train))
images, masks = batchs
im=images
k=11
images = images.to(device)
plt.figure(figsize=(20, 20))
out1=model1(images)
out1=out1.cpu().detach()
out2=model2(images)
out2=out2.cpu().detach()
out3=model3(images)
out3=out3.cpu().detach()

plt.subplot(1, 3, 1)
plt.xticks([])
plt.yticks([])
plt.imshow(im[k][1])
plt.title('Original image')

plt.subplot( 1, 3, 2)
plt.xticks([])
plt.yticks([])

plt.imshow(masks[k][0])
plt.title('Mask (Ground Truth)')

plt.subplot( 1, 3, 3)
plt.xticks([])
plt.yticks([])
plt.imshow(im[k][1])
plt.imshow(masks[k][0],alpha=0.2)
plt.title('Both')
plt.tight_layout()
plt.show()

plt.figure(figsize=(20, 20))
plt.subplot( 1, 3, 1)
plt.xticks([])
plt.yticks([])
# plt.imshow(im[k][1])
plt.imshow(out1[k][0])
plt.title('Mask predicted by CNN')

plt.subplot( 1, 3, 2)
plt.xticks([])
plt.yticks([])
# plt.imshow(im[k][1])
plt.imshow(out2[k][0])
plt.title('Mask predicted by UNet')

plt.subplot( 1, 3, 3)
plt.xticks([])
plt.yticks([])
# plt.imshow(im[k][1])
plt.imshow(out3[k][0])
plt.title('Mask predicted by AttUNet')

plt.tight_layout()
plt.show()